In [31]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\general_functions\Extracting Data Functions.ipynb"

In [32]:
f_name = os.path.join(os.path.dirname(os.getcwd()),"Edgewell Germany Dataset.xlsx")

client_manuf = ["Ecb"] 
client_brands = ["Isana Ecb"] 
prodORitem="Product"
decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 
categories =["Male Shaving"]
sectors = []
segments = ["Male Dispo","Male Blades","Male Razor"]
subsegments= []
subcategories= ["Male 1-Blade","Male 2-Blade","Male 3-Blade","Male 4-Blade","Male 5-Blade","Male 6-Blade","Male Double Edge"]

national = False
customareas= ""
areas = ["RETAILER"]

regions_RET  = ["Rossmann"]
channels_RET = []
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Jan 2025"

end_date = "2025-02-01"

OpenEditData=True
ManufOrTopC ="Top Companies"
BrandOrTopB="Top Brands"

percent = 100000
percentstr="'00 000"

direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories))
                ,"Segment":list(zip(["Sector"]*len(sectors),sectors))
                ,"SubSegment":list(zip(["Segment"]*len(sectors),sectors))
                ,"SubCategory":list(zip(["Segment"]*len(segments),segments))
                }


In [33]:
past_12_months = pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
past_3_months = pd.date_range(end=end_date , periods=3, freq='M').strftime('%b-%y').tolist()

In [34]:
f_path = Path.cwd()

excel = client.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True  # False
wb = excel.Workbooks.Open(f_name)
s_name = [s.Name for s in wb.Sheets][0]
ws=wb.Sheets([s.Name for s in wb.Sheets][0])
s_name = [s.Name for s in wb.Sheets][0]
pvtTable = ws.PivotTables(1)

#change report layout
pvtTable.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form

#change pivot table style
#Select from Design tab, try out Medium9 or Medium3
pvtTable.TableStyle2 = "pivotStyleMedium21"
pvtTable.ClearTable()

pvtTable.TableRange2.Cut(ws.Range("A16"))


fieldsNamePosition={}
for i in range(1,pvtTable.CubeFields.Count+1):
    fieldsNamePosition[str(pvtTable.CubeFields(i))]=i
    

In [25]:
# filter_dictionary_keys(fieldsNamePosition, 'Gross Profit')

In [26]:
start_time = time.time()

### Slide 67

In [7]:
brands_client_dfs = {}

In [8]:
row_list=['[Products].[Top Brands]']
column_list=[]
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Calendar].[MonthYear]', '[Products].[Sector]','[Products].[Segment]',f'[Products].[{ManufOrTopC}]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Gross Margin %]','[Measures].[Gross Margin % IYA]','[Measures].[Gross Profit]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Net Sales/Kg IYA]', '[Measures].[Volume Sales]','[Measures].[Trade Margin %]','[Measures].[Trade Profit]','[Measures].[Trade Profit/Kg]','[Measures].[Av Price/KG]','[Measures].[Value Sales IYA]','[Measures].[Value Sales]','[Measures].[Total WoB %]']


pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 


if len(client_manuf)>1:
    pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True    
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
else:
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"


calculate_category_data(brands_client_dfs,f' | {client_manuf[0]}')
calculate_sector_segment_data(brands_client_dfs,f'{client_manuf[0]} | ')

with open('Financials Datasets/brands_client_dfs.pickle', 'wb') as handle:
    pickle.dump(brands_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### DataFrames_By Markets

### Slide 67

In [9]:
retailers_P12M_dfs = {}
retailers_P3M_dfs = {}

if "RETAILER" in areas:
    row_list=[]
    if len(regions_RET)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_RET)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_RET)!=0:
        row_list.append('[Market].[Market]')
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Calendar].[MonthYear]',f'[Products].[{ManufOrTopC}]','[Products].[Top Brands]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=[]


    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

    row_list=[]
    column_list=[]
    filter_list=[]
    value_list=['[Measures].[Gross Margin %]','[Measures].[Gross Margin % IYA]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Net Sales/Kg IYA]','[Measures].[Value Sales IYA]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_sec_seg_subseg_subcat_Manufacturer(retailers_P12M_dfs)


    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_3_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

    calculate_sec_seg_subseg_subcat_Manufacturer(retailers_P3M_dfs)


with open('Financials Datasets/retailers_P12M_dfs.pickle', 'wb') as handle:
    pickle.dump(retailers_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Financials Datasets/retailers_P3M_dfs.pickle', 'wb') as handle:
    pickle.dump(retailers_P3M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
channel_P12M_dfs = {}
channel_P3M_dfs = {}

if "CHANNEL" in areas:
    row_list=[]
    if len(regions_CHAN)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CHAN)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CHAN)!=0:
        row_list.append('[Market].[Market]')
        
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Calendar].[MonthYear]',f'[Products].[{ManufOrTopC}]','[Products].[Top Brands]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=[]

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'

    row_list=[]
    column_list=[]
    filter_list=[]
    value_list=['[Measures].[Gross Margin %]','[Measures].[Gross Margin % IYA]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Net Sales/Kg IYA]','[Measures].[Value Sales IYA]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_sec_seg_subseg_subcat_Manufacturer(channel_P12M_dfs)

    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_3_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

    calculate_sec_seg_subseg_subcat_Manufacturer(channel_P3M_dfs)


with open('Financials Datasets/channel_P12M_dfs.pickle', 'wb') as handle:
    pickle.dump(channel_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Financials Datasets/channel_P3M_dfs.pickle', 'wb') as handle:
    pickle.dump(channel_P3M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

FileNotFoundError: [Errno 2] No such file or directory: 'Financials Datasets/channel_P12M_dfs.pickle'

In [11]:
cust_P12M_dfs={}
cust_P3M_dfs={}

if customareas in areas:

    row_list=[]
    if len(regions_CUST)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CUST)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CUST)!=0:
        row_list.append('[Market].[Market]')

    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Calendar].[MonthYear]',f'[Products].[{ManufOrTopC}]','[Products].[Top Brands]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=[]

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 


    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'


    row_list=[]
    column_list=[]
    filter_list=[]
    value_list=['[Measures].[Gross Margin %]','[Measures].[Gross Margin % IYA]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Net Sales/Kg IYA]','[Measures].[Value Sales IYA]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_sec_seg_subseg_subcat_Manufacturer(cust_P12M_dfs)


    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_3_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

    calculate_sec_seg_subseg_subcat_Manufacturer(cust_P3M_dfs)


with open('Financials Datasets/cust_P12M_dfs.pickle', 'wb') as handle:
    pickle.dump(cust_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)        
with open('Financials Datasets/cust_P3M_dfs.pickle', 'wb') as handle:
    pickle.dump(cust_P3M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### DataFrames By_Sectors

### Slide 68 & 69 & 71 & 72

In [15]:
for ke, val in direct_parent.items():
    globals()[f"{ke}_P12M_client_dfs"] = {}
    globals()[f"{ke}_P3M_client_dfs"] = {}
 
    row_list=[f'[Products].[{ke}]'] 
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Calendar].[MonthYear]',f'[Products].[{ManufOrTopC}]','[Products].[Top Brands]']
    value_list=[]

    
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True    
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"

    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 
    value_list=['[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Trade Margin %]','[Measures].[Trade Margin % DYA]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Gross Profit/Kg]','[Measures].[Gross Margin %]','[Measures].[Trade Profit IYA]','[Measures].[VSOD IYA]','[Measures].[Av Price/KG]','[Measures].[VAT/Kg]','[Measures].[Total COGS/Kg]','[Measures].[Trade Profit/Kg]','[Measures].[Rate of Sales]','[Measures].[WD]','[Measures].[Total WoB %]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_category_data(globals()[f"{ke}_P12M_client_dfs"],f' | {client_manuf[0]}')

    row_list=[f'[Products].[{ke}]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Calendar].[MonthYear]',f'[Products].[{ManufOrTopC}]','[Products].[Top Brands]']
    value_list=[]
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True    
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"


    # pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_3_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 
    value_list=['[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Trade Margin %]','[Measures].[Trade Margin % DYA]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Gross Profit/Kg]','[Measures].[Gross Margin %]','[Measures].[Trade Profit IYA]','[Measures].[VSOD IYA]','[Measures].[Av Price/KG]','[Measures].[VAT/Kg]','[Measures].[Total COGS/Kg]','[Measures].[Trade Profit/Kg]','[Measures].[Rate of Sales]','[Measures].[WD]','[Measures].[Total WoB %]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    calculate_category_data(globals()[f"{ke}_P3M_client_dfs"] ,f' | {client_manuf[0]}')

    with open(f'Financials Datasets/{ke}_P12M_client_dfs.pickle', 'wb') as handle:
        pickle.dump(globals()[f"{ke}_P12M_client_dfs"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'Financials Datasets/{ke}_P3M_client_dfs.pickle', 'wb') as handle:
        pickle.dump( globals()[f"{ke}_P3M_client_dfs"] , handle, protocol=pickle.HIGHEST_PROTOCOL)   

### Slide 70 &73 &74

In [27]:
product_P12M_dfs={}

row_list=[ '[Products].[Top Brands]',f'[Products].[{prodORitem}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubCategory]','[Products].[SubSegment]',f'[Products].[{ManufOrTopC}]']
value_list=['[Measures].[WD]','[Measures].[Net Sales/Kg]','[Measures].[Net Sales]','[Measures].[Net Sales/Unit]','[Measures].[Gross Profit/Unit]','[Measures].[Gross Margin %]','[Measures].[Trade Margin %]','[Measures].[Av Price/Unit]','[Measures].[VAT/Unit]','[Measures].[Trade Profit/Unit]','[Measures].[Total COGS/Unit]',f'[Measures].[{prodORitem} Sales Rate]','[Measures].[Value Sales IYA]']
pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_12_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

if len(client_manuf)>1:
    pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
else:
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"


pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

calculate_category_data(product_P12M_dfs,f' | {client_manuf[0]}')
calculate_sector_segment_data(product_P12M_dfs,f'{client_manuf[0]} | ')


with open('Financials Datasets/product_P12M_dfs.pickle', 'wb') as handle:
    pickle.dump(product_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)        

In [28]:
product_P3M_dfs={}

row_list=[ '[Products].[Top Brands]',f'[Products].[{prodORitem}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]', '[Products].[SubSegment]', '[Products].[SubCategory]',f"[Products].[{ManufOrTopC}]"]
value_list=['[Measures].[Net Sales]','[Measures].[WD]',f'[Measures].[{prodORitem} Sales Rate]','[Measures].[Value Sales IYA]','[Measures].[Gross Profit/Unit]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in past_3_months:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 


if len(client_manuf)>1:
    pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
else:
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"


pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

calculate_category_data(product_P3M_dfs,f' | {client_manuf[0]}')
calculate_sector_segment_data(product_P3M_dfs,f'{client_manuf[0]} | ')


with open('Financials Datasets/product_P3M_dfs.pickle', 'wb') as handle:
    pickle.dump(product_P3M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Slide 75

In [29]:
for ke, val in direct_parent.items():
    globals()[f"manuf_{ke}_dfs"] = {}
    
    row_list=[ f'[Products].[{ManufOrTopC}]',f'[Products].[{ke}]']
  
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Calendar].[MonthYear]']
    value_list=[]
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in past_12_months:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 
    value_list=['[Measures].[Volume Sales]','[Measures].[Value Share]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Trade Margin %]','[Measures].[Net Sales]','[Measures].[Net Sales/Kg]','[Measures].[Gross Profit/Kg]','[Measures].[Gross Margin %]','[Measures].[Gross to Net IYA]','[Measures].[VSOD IYA]','[Measures].[Av Price/KG]','[Measures].[VAT/Kg]','[Measures].[Total COGS/Kg]','[Measures].[Trade Profit/Kg]','[Measures].[Rate of Sales]','[Measures].[WD]','[Measures].[Total WoB %]','[Measures].[Trade Profit]','[Measures].[Gross Profit]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_category_data(globals()[f"manuf_{ke}_dfs"] )
    
    with open(f'Financials Datasets/manuf_{ke}_dfs.pickle', 'wb') as handle:
        pickle.dump( globals()[f"manuf_{ke}_dfs"] , handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

#40 mins

Script started at: Wed May  7 17:32:56 2025
Script ended at: Wed May  7 17:38:03 2025
Elapsed time: 307.54 seconds
